# Metric Analysis for ECON model

In [1]:
%reload_ext autoreload
%autoreload 2

# from common.metrics.gradient import Gradient
import torch
import torchinfo
import os
import sys

# import modules from ECON model
module_path = os.path.abspath(os.path.join('../../workspace/models/econ/code/')) # or the path to your source code
sys.path.insert(0, module_path)
from q_autoencoder import AutoEncoder
from autoencoder_datamodule import AutoEncoderDataModule

# import modules from common metrics
module_path = os.path.abspath(os.path.join('../../workspace/common/metrics/')) # or the path to your source code
sys.path.insert(0, module_path)
from gradient import Gradient



/opt/conda/envs/loss_landscape/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-11-20 21:48:15.975474: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-20 21:48:16.694693: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/opt/conda/envs/loss_landscape/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


Select the desired ECON model:

In [2]:
base_path = "/loss_landscape/checkpoint/different_knobs_subset_10/"
batch_size = 1024
learning_rate = 0.1
precision = 8
size = 'small'

In [3]:
def load_model(path, batch_size, learning_rate, precision, size):
    model_path = path + f'bs{batch_size}_lr{learning_rate}/ECON_{precision}b/{size}/net_1_best.pkl'
    model = AutoEncoder(
        quantize=(precision < 32),
        precision=[
            precision,
            precision,
            precision+3
        ],
        learning_rate=learning_rate,
        econ_type=size
    )
    model(torch.randn((1, 1, 8, 8)))  # Update tensor shapes 
    print(model_path)
    model_param = torch.load(model_path)
    model.load_state_dict(model_param['state_dict'])
    return model


model = load_model(base_path, batch_size, learning_rate, precision, size)
torchinfo.summary(model, input_size=(1, 1, 8, 8))  # (B, C, H, W)            

/loss_landscape/checkpoint/different_knobs_subset_10/bs1024_lr0.1/ECON_8b/small/net_1_best.pkl


Layer (type:depth-idx)                   Output Shape              Param #
AutoEncoder                              [1, 1, 8, 8]              --
├─QuantizedEncoder: 1-1                  [1, 16]                   --
│    └─QuantAct: 2-1                     [1, 1, 8, 8]              --
│    └─QuantConv2d: 2-2                  [1, 1, 4, 4]              20
│    └─ReLU: 2-3                         [1, 1, 4, 4]              --
│    └─QuantAct: 2-4                     [1, 1, 4, 4]              --
│    └─Flatten: 2-5                      [1, 16]                   --
│    └─QuantLinear: 2-6                  [1, 16]                   272
│    └─ReLU: 2-7                         [1, 16]                   --
├─Sequential: 1-2                        [1, 1, 8, 8]              --
│    └─Linear: 2-8                       [1, 128]                  2,176
│    └─ReLU: 2-9                         [1, 128]                  --
│    └─Unflatten: 2-10                   [1, 8, 4, 4]              --
│    └─Conv

Setup the data loader:

In [4]:
data_path = '../../data/ECON/Elegun'
processed_file = 'nELinks5.npy'

data_module = AutoEncoderDataModule(
    data_dir=data_path,
    data_file=os.path.join(data_path, processed_file),
    batch_size=batch_size,
    num_workers=4
)
# checek if we have processed the data
if not os.path.exists(os.path.join(data_path, processed_file)):
    print('Processing the data...')
    data_module.process_data(save=True)

data_module.setup(0)

data_loader = data_module.val_dataloader()

Loaded shaped data shape: (1740950, 1, 8, 8)
Loaded shaped data datatype: float32


In [8]:
import pytorch_lightning as pl 
import tqdm
import numpy as np
import multiprocessing
import time
module_path = os.path.abspath(os.path.join('../../workspace/models/econ/code/')) # or the path to your source code
sys.path.insert(0, module_path)
from utils_pt import *



trainer = pl.Trainer(
        max_epochs=5,
        accelerator="auto",
        devices=1
    )

_, val_sum = data_module.get_val_max_and_sum()
model.set_val_sum(val_sum)
data_module.setup("test")

test_results_new = trainer.test(model, dataloaders=data_module.test_dataloader())

# test_results_old = test_model(model, data_module.test_dataloader())

test_results_new


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Reading files ['5Elinks_data.csv', 'nELinks5.npy']
Input data shape: (1740950, 48)


You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Prepped shaped data shape: (1740950, 1, 8, 8)
Loaded shaped data shape: (1740950, 1, 8, 8)
Loaded shaped data datatype: float32
Testing DataLoader 0: 100%|██████████| 341/341 [00:32<00:00, 10.46it/s]

AssertionError: 
Arrays are not almost equal to 7 decimals
a and b vector must have the same sum
Mismatched elements: 1 / 1 (100%)
Max absolute difference: 1.
Max relative difference: inf
 x: array(1.)
 y: array([0.])

Model performances:

In [ ]:
%load_ext tensorboard
%tensorboard --logdir base_path + f'bs{batch_size}_lr{learning_rate}/ECON_{precision}b/{size}/'

Launching TensorBoard...

## Gradient computation

In [ ]:
metric = Gradient(
    model=model,
    data_loader=data_loader,
    loss=model.loss
)

metric.compute()

Computing the gradients...


ValueError: too many values to unpack (expected 2)